In [1]:
from pynq import Overlay
from pynq import Xlnk
import numpy as np

In [2]:
chin = 3
chout = 2
kx = 2
ky = 2
win = 3
hin = 3
stride = 1
padding = 0
hout = 2
wout = 2

In [8]:
# Load the overlay
overlay = Overlay("design_1.bit")
conv = overlay.conv_0
overlay.is_loaded()

True

In [9]:
xlnk = Xlnk()
# Allocate memory for input data
feature_in = xlnk.cma_array(shape=(3, 3, 3), cacheable=0, dtype=np.float32)
weight = xlnk.cma_array(shape=(2, 3, 2, 2), cacheable=0, dtype=np.float32)
bias = xlnk.cma_array(shape=(2), dtype=np.float32)
feature_out = xlnk.cma_array(shape=(2, 2, 2), dtype=np.float32)

In [37]:
# Assign values to input data
feature_in[:] = np.array([
    [[1, 2.5, 1], 
     [1, 1, 1], 
     [1, 1, 1]],
    
    [[2, 2, 2], 
     [2, 2, 2], 
     [2, 2, 2]],
    
    [[1, 1, 1], 
     [1, 1, 1], 
     [1, 1, 1]]
], dtype=np.float32)

weight[:] = np.array([
    [
        [[1, 1], 
         [1, 1]],
        
        [[1, 1],
         [1, 1]],
        
        [[1, 1],
         [1, 1]],
    ],
    [
        
        [[2, 2],
         [2, 2]],

        [[2, 2], 
         [2, 2]],
        
        [[2, 2],
         [2, 2]]
    ]
], dtype=np.float32)

bias[:] = np.array([1.5, 1], dtype=np.float32)

In [38]:
def Run_Conv(conv_ip, chin, chout, kx, ky, win, hin, stride, padding, feature_in, weight, bias, feature_out):
    conv_ip.write(0x10, chin)
    conv_ip.write(0x18, chout)
    conv_ip.write(0x20, kx)
    conv_ip.write(0x28, ky)
    conv_ip.write(0x30, win)
    conv_ip.write(0x38, hin)
    conv_ip.write(0x40, stride)
    conv_ip.write(0x48, padding)
    conv_ip.write(0x50, feature_in.physical_address)
    conv_ip.write(0x58, weight.physical_address)
    conv_ip.write(0x68, bias.physical_address)
    conv_ip.write(0x60, feature_out.physical_address)
    conv_ip.write(0, (conv.read(0)&0x80)|0x01 ) #start conv IP
    while True:
        if conv_ip.read(0x00) & 0x2:  # 检查 ap_done 位
            print('done')
            break


In [39]:
# Run convolution
Run_Conv(conv, chin, chout, kx, ky, win, hin, stride, padding, feature_in, weight, bias, feature_out)

print("Convolution result:")
print(feature_out)

done
Convolution result:
[[[ 19.   19. ]
  [ 17.5  17.5]]

 [[ 36.   36. ]
  [ 33.   33. ]]]
